In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.year.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Year'] = df_input['Year'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：69



,Year
0,2025
1,2024
2,2023
3,2022
4,2021
...,...
64,1961
65,1960
66,1959
67,1958


In [3]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Year    2025
 Name: 0, dtype: int64)

In [4]:
import time

data = {'action': 'bf_ajax_hook',
        'type': 'make',
        'c_year': crawler_series['Year']}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = 'https://www.rainx.com/'

        resp = requests.post('https://www.rainx.com/wp-admin/admin-ajax.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            with open('./make.html', 'w', encoding='UTF-8') as file:
                file.write(str(soup))

            if html.xpath('//option/text()')[0].strip() == 'Make':
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

print(soup.prettify())

1
<html>
 <body>
  <option value="">
   Make
  </option>
  <option value="Cadillac">
   Cadillac
  </option>
  <option value="INFINITI">
   INFINITI
  </option>
  <option value="Toyota">
   Toyota
  </option>
  0
 </body>
</html>



In [5]:
list_make = [text for text in html.xpath('//option/text()')[1:]]

list_make

['Cadillac', 'INFINITI', 'Toyota']

In [6]:
list_dict_correct.extend([{'Year': crawler_series['Year'],
                           'Make No': i + 1,
                           'Make': make} for i, make in enumerate(list_make)])

list_dict_correct

[{'Year': np.int64(2025), 'Make No': 1, 'Make': 'Cadillac'},
 {'Year': np.int64(2025), 'Make No': 2, 'Make': 'INFINITI'},
 {'Year': np.int64(2025), 'Make No': 3, 'Make': 'Toyota'}]

In [7]:
crawler_status = True

crawler_status

True

In [8]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[Year                           2025
 Check_Url    https://www.rainx.com/
 Name: 0, dtype: object]

In [9]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Year', 'Make No'],
                                                             ascending=[False, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_make.xlsx', index=False)

df_correct

,Year,Make No,Make
0,2025,1,Cadillac
1,2025,2,INFINITI
2,2025,3,Toyota


In [10]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Year'],
                                                           ascending=[False],
                                                           ignore_index=True)
    df_error.to_excel('./test_make_error.xlsx', index=False)

df_error

,Year,Check_Url
0,2025,https://www.rainx.com/
